In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r"C:\Users\Kieran\Documents\Career Foundry\Data Immersion\4.0 Python\Instacart Basket Analysis"

In [3]:
# Import data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_new_variables.pkl'))

In [4]:
# first million rows
df = ords_prods_merge[:1000000]

In [5]:
df.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest day,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,0.0,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders


In [6]:
df.shape

(1000000, 18)

# Group by

1) Split data in to groups based on some criteria  
2) Apply a function to each group  
3) Combine results in to df or other data structure or new column  

In [7]:
# this creates an object, but by itself the object will not be visible. Step 1.
df.groupby('product_name')

# Aggregating with agg()

Getting a single descriptive statistic (mean) for order_number grouped by department_id. This will allow us to compare average number of orders across departments. 

In [8]:
# split the data, then apply agg()
df.groupby('department_id').agg({'order_number' : ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


we dont need the agg() function to calculate mean here

In [9]:
#note the different syntax
df.groupby('department_id')['order_number'].mean()

department_id
4     18.825780
7     17.472355
13    17.993423
14    19.246334
16    19.463012
17    11.294069
19    19.305237
20    17.599636
Name: order_number, dtype: float64

Multiple aggregations just require adding more arguments

In [10]:
df.groupby('department_id').agg({'order_number' : ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
4                18.825780   1  99
7                17.472355   1  99
13               17.993423   1  99
14               19.246334   1  99
16               19.463012   1  99
17               11.294069   1  98
19               19.305237   1  99
20               17.599636   1  99

# Aggregate data with transform

1) Split the data in to groups based on user_id  
2) Apply transform() to order_number column to get max order for each user  
3) Results in to new column  

In [11]:
#create new max_order column that contains max order number for each user. 
ords_prods_merge['max order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [12]:
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest day,busiest_days,busiest_period_of_day,max order
0,2539329,1,1,2,8,0.0,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Average orders,10
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Slowest days,Most orders,10
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Average orders,10
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Slowest days,Most orders,10


Note that within the max_order column, each number will appear the same number of times as its value - because it appears once for each time that user made an order. E.G a user makes 10 orders, therefore their max order number is 10, it appears alongside each order placed - 10 times.

# Deriving columns with loc()

As in previous exercise, I will use loc() to create a flag for loyal customer (40+ orders), regular customer (10 >= 40 order), new customer (< 10 orders)

In [22]:
ords_prods_merge.loc[ords_prods_merge['max order'] <= 10, 'loyalty_flag'] = 'New Customer'

In [19]:
ords_prods_merge.loc[(ords_prods_merge['max order'] > 10) & (ords_prods_merge['max order'] <= 40), 'loyalty_flag'] = 'Regular Customer'

In [20]:
ords_prods_merge.loc[ords_prods_merge['max order'] < 40, 'loyalty_flag'] = 'Loyal Customer'

In [23]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Loyal Customer      15509465
NaN                 10284093
New Customer         6243990
Regular Customer      367311
Name: loyalty_flag, dtype: int64

In [25]:
# returning head of only 3 columns
ords_prods_merge[['user_id', 'loyalty_flag', 'max order']].head(50)

,user_id,loyalty_flag,max order
0,1,New Customer,10
1,1,New Customer,10
2,1,New Customer,10
3,1,New Customer,10
4,1,New Customer,10
5,1,New Customer,10
6,1,New Customer,10
7,1,New Customer,10
8,1,New Customer,10
9,1,New Customer,10
